# 🎮 Sokoban Supervised Learning with PoT

This notebook trains a **Pondering over Thoughts (PoT)** model on Sokoban puzzles using **supervised learning** - identical to our Sudoku training pipeline.

## What is Sokoban?

**Sokoban** (倉庫番, "warehouse keeper") is a classic puzzle game where you push boxes onto target locations.

<img src="https://upload.wikimedia.org/wikipedia/commons/4/4b/Sokoban_ani.gif" width="300" alt="Sokoban gameplay animation">

**Rules:**
- 🧑 Player can move in 4 directions (up/down/left/right)
- 📦 Player can push ONE box at a time (not pull)
- 🎯 Goal: Push ALL boxes onto target squares
- ⚠️ Boxes can get stuck in corners (deadlock = game over!)

▶️ **Watch gameplay:** [Sokoban Tutorial on YouTube](https://www.youtube.com/watch?v=4SjXQ_bHTxU)

**Why is it hard for AI?**
- PSPACE-complete (exponential state space)
- Sparse rewards (only get reward when solved)
- Long-horizon planning required
- Easy to create unsolvable states

## Benchmark Comparison

| Method | Simple (6×6, 1 box) | Complex (10×10, 2 boxes) | Notes |
|--------|---------------------|--------------------------|-------|
| SFT (paper) | ~50% | ~15% | Supervised fine-tuning |
| GPT-4 + LangGraph | varies | varies | [Blog](https://blog.gopenai.com/using-llms-and-langgraph-to-tackle-sokoban-puzzles-5f50b43b9515) |
| RL (PPO) | ~20% | <5% | Very hard to train |
| Random | 25% | 25% | 4 actions |
| **PoT (this notebook)** | TBD | TBD | Adaptive depth |

## Dataset

We use the [Xiaofeng77/sokoban](https://huggingface.co/datasets/Xiaofeng77/sokoban) HuggingFace dataset with:
- ~3,000 (board, optimal_action) pairs
- On-the-fly augmentation (8x via rotations/flips)
- Cross-entropy loss + Q-halt loss (identical to Sudoku)


In [ ]:
# @title 🔧 Setup (Run First)
# @markdown Install dependencies and clone repository

!pip install -q torch datasets tqdm wandb gym-sokoban

# Clone PoT repository
!git clone -q https://github.com/ebenartzy/PoT.git 2>/dev/null || (cd PoT && git pull -q)

import sys
sys.path.insert(0, 'PoT')

print("✅ Setup complete!")


In [ ]:
# @title 📊 Configuration
# @markdown ### Model Type
MODEL_TYPE = "hybrid_pot"  # @param ["pot", "hybrid_pot", "baseline"]
CONTROLLER_TYPE = "transformer"  # @param ["transformer", "gru", "lstm", "diffusion", "swin", "mamba"]

# @markdown ### Architecture
D_MODEL = 256  # @param {type:"slider", min:64, max:512, step:64}
D_FF = 512  # @param {type:"slider", min:128, max:2048, step:128}
N_HEADS = 4  # @param {type:"slider", min:2, max:16, step:2}
N_LAYERS = 2  # @param {type:"slider", min:1, max:8, step:1}
DROPOUT = 0.0  # @param {type:"slider", min:0.0, max:0.5, step:0.1}

# @markdown ### PoT Iteration Parameters
R = 4  # @param {type:"slider", min:1, max:16, step:1}
T = 4  # @param {type:"slider", min:1, max:8, step:1}

# @markdown ### Hybrid PoT Parameters (H/L cycles)
H_LAYERS = 2  # @param {type:"slider", min:1, max:4, step:1}
L_LAYERS = 2  # @param {type:"slider", min:1, max:4, step:1}
H_CYCLES = 2  # @param {type:"slider", min:1, max:8, step:1}
L_CYCLES = 6  # @param {type:"slider", min:1, max:16, step:1}

# @markdown ### Controller Parameters
D_CTRL = 128  # @param {type:"slider", min:32, max:256, step:32}
MAX_DEPTH = 128  # @param {type:"slider", min:32, max:256, step:32}

# @markdown ### Feature Injection
INJECTION_MODE = "broadcast"  # @param ["none", "broadcast", "film", "depth_token", "cross_attn", "alpha_gated"]
INJECTION_MEMORY_SIZE = 8  # @param {type:"slider", min:4, max:32, step:4}
INJECTION_N_HEADS = 4  # @param {type:"slider", min:2, max:8, step:2}

# @markdown ### ACT (Adaptive Computation Time) Parameters
HALT_MAX_STEPS = 4  # @param {type:"slider", min:1, max:16, step:1}
HALT_EXPLORATION_PROB = 0.1  # @param {type:"slider", min:0.0, max:1.0, step:0.05}
ALLOW_EARLY_HALT_EVAL = True  # @param {type:"boolean"}

# @markdown ### Training Hyperparameters
EPOCHS = 100  # @param {type:"slider", min:10, max:500, step:10}
BATCH_SIZE = 64  # @param {type:"slider", min:16, max:256, step:16}
LEARNING_RATE = 3e-4  # @param {type:"number"}
WEIGHT_DECAY = 0.01  # @param {type:"number"}
GRAD_CLIP = 1.0  # @param {type:"slider", min:0.1, max:5.0, step:0.1}
WARMUP_STEPS = 100  # @param {type:"slider", min:0, max:1000, step:50}
LR_MIN_RATIO = 0.1  # @param {type:"slider", min:0.01, max:1.0, step:0.01}
BETA1 = 0.9  # @param {type:"number"}
BETA2 = 0.95  # @param {type:"number"}

# @markdown ### HRM Gradient Style
HRM_GRAD_STYLE = True  # @param {type:"boolean"}

# @markdown ### Data
AUGMENT = True  # @param {type:"boolean"}
SEED = 42  # @param {type:"integer"}
N_GENERATED = 0  # @param {type:"slider", min:0, max:10000, step:500}
GEN_DIFFICULTY = "simple"  # @param ["simple", "larger", "two_boxes", "complex"]

# @markdown ### Logging
USE_WANDB = False  # @param {type:"boolean"}
WANDB_PROJECT = "sokoban-pot"  # @param {type:"string"}

# @markdown ### Evaluation
EVAL_DIFFICULTIES = ["simple", "complex"]  # Easy (6x6,1box) and Hard (10x10,2boxes)
EVAL_SAMPLES = 200  # @param {type:"slider", min:50, max:500, step:50}

# Build config dict for easy access
CONFIG = {
    # Model
    'model_type': MODEL_TYPE,
    'controller_type': CONTROLLER_TYPE,
    'd_model': D_MODEL,
    'd_ff': D_FF,
    'n_heads': N_HEADS,
    'n_layers': N_LAYERS,
    'dropout': DROPOUT,
    # PoT iterations
    'R': R,
    'T': T,
    # Hybrid H/L cycles
    'H_layers': H_LAYERS,
    'L_layers': L_LAYERS,
    'H_cycles': H_CYCLES,
    'L_cycles': L_CYCLES,
    # Controller
    'd_ctrl': D_CTRL,
    'max_depth': MAX_DEPTH,
    # Feature Injection
    'injection_mode': INJECTION_MODE,
    'injection_memory_size': INJECTION_MEMORY_SIZE,
    'injection_n_heads': INJECTION_N_HEADS,
    # ACT
    'halt_max_steps': HALT_MAX_STEPS,
    'halt_exploration_prob': HALT_EXPLORATION_PROB,
    'allow_early_halt_eval': ALLOW_EARLY_HALT_EVAL,
    # Training
    'epochs': EPOCHS,
    'batch_size': BATCH_SIZE,
    'lr': LEARNING_RATE,
    'weight_decay': WEIGHT_DECAY,
    'grad_clip': GRAD_CLIP,
    'warmup_steps': WARMUP_STEPS,
    'lr_min_ratio': LR_MIN_RATIO,
    'beta1': BETA1,
    'beta2': BETA2,
    # HRM
    'hrm_grad_style': HRM_GRAD_STYLE,
    # Data
    'augment': AUGMENT,
    'seed': SEED,
    'n_generated': N_GENERATED,
    'gen_difficulty': GEN_DIFFICULTY,
}

print(f"Config: {MODEL_TYPE} ({CONTROLLER_TYPE})")
print(f"  Architecture: d={D_MODEL}, ff={D_FF}, heads={N_HEADS}, layers={N_LAYERS}")
print(f"  PoT: R={R}, T={T}")
print(f"  Hybrid: H_cycles={H_CYCLES}, L_cycles={L_CYCLES}")
print(f"  Controller: d_ctrl={D_CTRL}, max_depth={MAX_DEPTH}")
print(f"  Injection: mode={INJECTION_MODE}")
print(f"  ACT: halt_max={HALT_MAX_STEPS}")
print(f"  Training: epochs={EPOCHS}, lr={LEARNING_RATE}, batch={BATCH_SIZE}")


In [ ]:
# @title 📥 Load Dataset
# @markdown Downloads Sokoban dataset from HuggingFace + optional generated data
# @markdown **Important:** Train/Val/Test are kept PURE (no augmentation leakage)

import torch
from torch.utils.data import DataLoader, Subset
import numpy as np

from src.data.sokoban_hf import SokobanHFDataset, SokobanCombinedDataset

# IMPORTANT: Load WITHOUT augmentation first to split cleanly
# Then apply augmentation only to training samples
print("Loading datasets...")

if N_GENERATED > 0:
    print(f"  HuggingFace + generating {N_GENERATED} additional {GEN_DIFFICULTY} puzzles...")
    # Load without augmentation for clean split
    full_ds_no_aug = SokobanCombinedDataset(
        hf_split="train",
        n_generated=N_GENERATED,
        difficulty=GEN_DIFFICULTY,
        augment=False,  # No augmentation for splitting
        seed=SEED,
    )
    # Load with augmentation for training
    full_ds_aug = SokobanCombinedDataset(
        hf_split="train",
        n_generated=N_GENERATED,
        difficulty=GEN_DIFFICULTY,
        augment=AUGMENT,
        seed=SEED,
    )
else:
    # Load without augmentation for clean split
    full_ds_no_aug = SokobanHFDataset(split="train", augment=False)
    # Load with augmentation for training
    full_ds_aug = SokobanHFDataset(split="train", augment=AUGMENT)

# Test set: completely separate (from HuggingFace 'test' split)
test_ds = SokobanHFDataset(split="test", augment=False)

# Split indices (not datasets!) to keep train/val PURE
n_total = len(full_ds_no_aug)
val_size = min(500, n_total // 5)
train_size = n_total - val_size

# Deterministic shuffle
rng = np.random.default_rng(SEED)
indices = rng.permutation(n_total)
train_indices = indices[:train_size].tolist()
val_indices = indices[train_size:].tolist()

# Train: uses augmented dataset (on-the-fly augmentation)
# Val: uses non-augmented dataset (PURE - no augmentation)
train_subset = Subset(full_ds_aug, train_indices)
val_subset = Subset(full_ds_no_aug, val_indices)

train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

print(f"\n✅ Data splits (PURE - no leakage):")
print(f"   Train: {len(train_subset)} samples (augment={'ON' if AUGMENT else 'OFF'})")
print(f"   Val:   {len(val_subset)} samples (augment=OFF, pure)")
print(f"   Test:  {len(test_ds)} samples (augment=OFF, separate HF split)")
print(f"   Board shape: {full_ds_no_aug.board_shape}")
if N_GENERATED > 0:
    print(f"   +{N_GENERATED} generated {GEN_DIFFICULTY} puzzles")


In [ ]:
# @title 🏗️ Create Model

from src.pot.models.sokoban_solver import PoTSokobanSolver, HybridPoTSokobanSolver, BaselineSokobanSolver

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(SEED)

H, W = full_ds_no_aug.board_shape
seq_len = H * W

# Controller kwargs for advanced controllers
controller_kwargs = {
    'n_heads': N_HEADS,
    'd_ctrl': D_CTRL,
    'max_depth': MAX_DEPTH,
}

# Injection kwargs for cross_attn mode
injection_kwargs = {
    'memory_size': INJECTION_MEMORY_SIZE,
    'n_heads': INJECTION_N_HEADS,
} if INJECTION_MODE == 'cross_attn' else None

if MODEL_TYPE == "pot":
    model = PoTSokobanSolver(
        board_height=H,
        board_width=W,
        d_model=D_MODEL,
        n_heads=N_HEADS,
        n_layers=N_LAYERS,
        d_ff=D_FF,
        dropout=DROPOUT,
        R=R,
        controller_type=CONTROLLER_TYPE,
        controller_kwargs=controller_kwargs,
        max_depth=MAX_DEPTH,
    )
elif MODEL_TYPE == "hybrid_pot":
    model = HybridPoTSokobanSolver(
        board_height=H,
        board_width=W,
        d_model=D_MODEL,
        n_heads=N_HEADS,
        H_layers=H_LAYERS,
        L_layers=L_LAYERS,
        d_ff=D_FF,
        dropout=DROPOUT,
        H_cycles=H_CYCLES,
        L_cycles=L_CYCLES,
        T=T,
        halt_max_steps=HALT_MAX_STEPS,
        halt_exploration_prob=HALT_EXPLORATION_PROB,
        allow_early_halt_eval=ALLOW_EARLY_HALT_EVAL,
        hrm_grad_style=HRM_GRAD_STYLE,
        controller_type=CONTROLLER_TYPE,
        controller_kwargs=controller_kwargs,
        injection_mode=INJECTION_MODE,
        injection_kwargs=injection_kwargs,
    )
else:
    model = BaselineSokobanSolver(
        board_height=H,
        board_width=W,
        d_model=D_MODEL,
        n_heads=N_HEADS,
        n_layers=N_LAYERS,
        d_ff=D_FF,
        dropout=DROPOUT,
    )

model = model.to(device)
n_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"✅ {MODEL_TYPE} ({CONTROLLER_TYPE}) model created")
print(f"   Parameters: {n_params:,}")
print(f"   Device: {device}")
print(f"   Board: {H}x{W} = {seq_len} tokens")


In [ ]:
# @title 🚀 Train Model
# @markdown Supervised training with cross-entropy + Q-halt loss (identical to Sudoku)

from src.training.sokoban_supervised import train_supervised
import time

if USE_WANDB:
    import wandb
    run_name = f"{MODEL_TYPE}-{CONTROLLER_TYPE}-R{R}-H{H_CYCLES}L{L_CYCLES}"
    wandb.init(
        project=WANDB_PROJECT,
        name=run_name,
        config=CONFIG,
    )

print(f"Training {MODEL_TYPE} ({CONTROLLER_TYPE})...")
print(f"  Epochs: {EPOCHS}, LR: {LEARNING_RATE}, Batch: {BATCH_SIZE}")
print(f"  PoT: R={R}, T={T}, H_cycles={H_CYCLES}, L_cycles={L_CYCLES}")
print()

start_time = time.time()

results = train_supervised(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    epochs=EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    grad_clip=GRAD_CLIP,
    warmup_steps=WARMUP_STEPS,
    use_pot=(MODEL_TYPE != "baseline"),
    wandb_log=USE_WANDB,
    # Additional optimizer params
    betas=(BETA1, BETA2),
    lr_min_ratio=LR_MIN_RATIO,
)

train_time = time.time() - start_time

print(f"\n✅ Training complete!")
print(f"   Best val accuracy: {results['best_val_acc']:.2%}")
print(f"   Training time: {train_time / 60:.1f} min")


In [ ]:
# @title 📈 Evaluate on HuggingFace Test Set

from src.training.sokoban_supervised import evaluate

test_metrics = evaluate(model, test_loader, device)
print(f"\n📊 HuggingFace Test Set Results:")
print(f"   Accuracy: {test_metrics['accuracy']:.2%}")
print(f"   Loss: {test_metrics['loss']:.4f}")


In [ ]:
# @title 🎯 Multi-Difficulty Evaluation
# @markdown Evaluate on Simple (6×6, 1 box) and Complex (10×10, 2 boxes)

from src.data.sokoban_generator import SokobanGeneratedDataset

difficulty_results = {}

for difficulty in EVAL_DIFFICULTIES:
    print(f"\nGenerating {difficulty} test boards...")
    
    eval_ds = SokobanGeneratedDataset(
        difficulty=difficulty,
        n_samples=EVAL_SAMPLES,
        seed=1042,
        augment=False,
    )
    
    eval_loader = DataLoader(eval_ds, batch_size=BATCH_SIZE, shuffle=False)
    
    # Handle different board sizes
    eval_h, eval_w = eval_ds.board_shape
    if (eval_h, eval_w) != train_ds.board_shape:
        print(f"   ⚠️ Board size mismatch: train={train_ds.board_shape}, eval={eval_ds.board_shape}")
        print(f"   Skipping (model trained on fixed size)")
        difficulty_results[difficulty] = {'accuracy': None, 'note': 'size_mismatch'}
        continue
    
    eval_metrics = evaluate(model, eval_loader, device)
    difficulty_results[difficulty] = {
        'accuracy': eval_metrics['accuracy'],
        'loss': eval_metrics['loss'],
    }
    print(f"   {difficulty}: {eval_metrics['accuracy']:.2%} accuracy")

print("\n" + "=" * 50)
print("MULTI-DIFFICULTY RESULTS")
print("=" * 50)
for diff, res in difficulty_results.items():
    if res.get('accuracy') is not None:
        print(f"  {diff}: {res['accuracy']:.2%}")
    else:
        print(f"  {diff}: N/A ({res.get('note', 'error')})")


In [ ]:
# @title 📋 Final Comparison Table

simple_acc = difficulty_results.get('simple', {}).get('accuracy', 0) or 0
complex_acc = difficulty_results.get('complex', {}).get('accuracy', 0) or 0

print("""
╔══════════════════════════════════════════════════════════════════════════╗
║                    SOKOBAN BENCHMARK COMPARISON                          ║
╠══════════════════════════════════════════════════════════════════════════╣
║ Method                   │ Simple (6×6,1) │ Complex (10×10,2) │ Notes    ║
╠══════════════════════════╪════════════════╪═══════════════════╪══════════╣
║ SFT (paper)              │     ~50%       │      ~15%         │ Baseline ║
║ GPT-4 + LangGraph        │     varies     │      varies       │ Zero-shot║
║ RL (PPO)                 │     ~20%       │      <5%          │ Hard     ║
║ Random                   │      25%       │       25%         │ 4 actions║
╠══════════════════════════╪════════════════╪═══════════════════╪══════════╣""")

print(f"║ PoT (this run)           │    {simple_acc:5.1%}      │     {complex_acc:5.1%}        │ R={R}      ║")
print("╚══════════════════════════════════════════════════════════════════════════╝")

# Store ALL config + results for Optuna/hyperparameter search
COLAB_RESULTS = {
    # Full configuration (for Optuna)
    **CONFIG,
    # Results
    'best_val_acc': results['best_val_acc'],
    'hf_test_acc': test_metrics['accuracy'],
    'simple_acc': simple_acc,
    'complex_acc': complex_acc,
    'train_time_min': train_time / 60,
    'n_params': n_params,
    'difficulty_results': difficulty_results,
}

print(f"\n📊 Full Results (for Optuna search):")
print(f"   Model: {MODEL_TYPE} ({CONTROLLER_TYPE})")
print(f"   Architecture: d={D_MODEL}, ff={D_FF}, heads={N_HEADS}")
print(f"   PoT: R={R}, T={T}, H_cycles={H_CYCLES}, L_cycles={L_CYCLES}")
print(f"   ACT: halt_max={HALT_MAX_STEPS}, max_depth={MAX_DEPTH}")
print(f"   Val Acc: {results['best_val_acc']:.2%}")
print(f"   Test Acc: {test_metrics['accuracy']:.2%}")
print(f"   Simple Acc: {simple_acc:.2%}")
print(f"   Complex Acc: {complex_acc:.2%}")
print(f"   Params: {n_params:,}")
print(f"   Time: {train_time / 60:.1f} min")


In [ ]:
# @title 💾 Save Model & Results (Optional)

import os
import json

os.makedirs("checkpoints", exist_ok=True)

# Save model checkpoint
model_name = f"sokoban_{MODEL_TYPE}_{CONTROLLER_TYPE}_R{R}_H{H_CYCLES}L{L_CYCLES}"
save_path = f"checkpoints/{model_name}.pt"

torch.save({
    'model_state_dict': model.state_dict(),
    'config': CONFIG,
    'results': COLAB_RESULTS,
}, save_path)

print(f"✅ Model saved to {save_path}")

# Save results JSON (for Optuna aggregation)
results_path = f"checkpoints/{model_name}_results.json"
with open(results_path, 'w') as f:
    # Convert non-serializable items
    results_to_save = {k: v for k, v in COLAB_RESULTS.items() 
                       if not isinstance(v, dict) or k == 'difficulty_results'}
    json.dump(results_to_save, f, indent=2, default=str)

print(f"✅ Results saved to {results_path}")

# Print command to download
print(f"\n📥 Download command:")
print(f"   from google.colab import files")
print(f"   files.download('{save_path}')")
print(f"   files.download('{results_path}')")


## 📝 Notes

### Loss Function (Identical to Sudoku)
```python
loss = cross_entropy(action_logits, action_label) + 0.5 * bce(q_halt, is_correct)
```

### Configuration Parameters for Optuna Search

| Parameter | Description | Typical Range |
|-----------|-------------|---------------|
| **Architecture** | | |
| `D_MODEL` | Hidden dimension | 64-512 |
| `D_FF` | Feedforward dimension | 128-2048 |
| `N_HEADS` | Attention heads | 2-16 |
| `N_LAYERS` | Transformer layers | 1-8 |
| **PoT Iterations** | | |
| `R` | Refinement iterations | 1-16 |
| `T` | HRM period | 1-8 |
| **Hybrid H/L Cycles** | | |
| `H_CYCLES` | High-level (slow) cycles | 1-8 |
| `L_CYCLES` | Low-level (fast) cycles | 1-16 |
| `H_LAYERS` | Layers in H-level | 1-4 |
| `L_LAYERS` | Layers in L-level | 1-4 |
| **Controller** | | |
| `D_CTRL` | Controller hidden dimension | 32-256 |
| `MAX_DEPTH` | Max controller depth | 32-256 |
| **Feature Injection** | | |
| `INJECTION_MODE` | Injection mode | none/broadcast/film/etc |
| `INJECTION_MEMORY_SIZE` | Memory size (cross_attn) | 4-32 |
| `INJECTION_N_HEADS` | Heads (cross_attn) | 2-8 |
| **ACT (Adaptive Computation)** | | |
| `HALT_MAX_STEPS` | Max halting steps | 1-16 |
| `HALT_EXPLORATION_PROB` | Exploration probability | 0.0-1.0 |
| **Controller Types** | | |
| `transformer` | CausalDepthTransformerRouter | Default |
| `gru` | GRU-based controller | Fast |
| `lstm` | LSTM-based controller | |
| `diffusion` | Diffusion denoising | DiT-style |
| `swin` | Swin Transformer | Vision |
| `mamba` | Mamba SSM | State-space |
| **Data Generation** | | |
| `N_GENERATED` | Extra puzzles to generate | 0-10000 |
| `GEN_DIFFICULTY` | Difficulty of generated | simple/larger/two_boxes/complex |

### Difficulty Levels
| Difficulty | Size | Boxes | Avg Solution Length |
|------------|------|-------|---------------------|
| simple | 6×6 | 1 | ~4 moves |
| larger | 10×10 | 1 | ~15 moves |
| two_boxes | 6×6 | 2 | ~20 moves |
| complex | 10×10 | 2 | ~18 moves |

### Example Optuna Study
```python
import optuna

def objective(trial):
    R = trial.suggest_int('R', 1, 16)
    H_CYCLES = trial.suggest_int('H_cycles', 1, 8)
    L_CYCLES = trial.suggest_int('L_cycles', 1, 16)
    D_MODEL = trial.suggest_categorical('d_model', [128, 256, 512])
    # ... train and return complex_acc
    return complex_acc

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)
```

### References
- [Debunking SFT Generalization](https://arxiv.org/pdf/2510.00237)
- [LLMs + LangGraph for Sokoban](https://blog.gopenai.com/using-llms-and-langgraph-to-tackle-sokoban-puzzles-5f50b43b9515)
- [HuggingFace Dataset](https://huggingface.co/datasets/Xiaofeng77/sokoban)
